# 个人贷款违约预测
## 业务理解

* **银行贷款业务的核心目标**：降低客户贷款逾期率。
* **项目目的**：预测目前处于贷款期间的客户是否会违约
* **贷款违约的原因分析**：违约收益>违约成本
	1.**还款意愿不足**：违约收益大于违约成本。违约收益通常为贷款额度，是固定的，而违约成本是动态的，会基于客户的经济收入、社会地位等因素有所差异和波动。
		涉及业务阶段：贷前审核
	2.**还款能力不足**：经济条件恶化，如消费欲望超出了当前经济能力、生活状态不稳定，造成偿还能力不足。
		涉及业务阶段：贷后监控
* 确定**监测贷款违约的重要指标**：违约成本
	*	关于违约成本的衍生变量：
		* 一级衍生变量：资产余额
		* 二级衍生变量：资产余额均值(mean)，资产余额波动率(std)
		* 三级衍生变量：资产余额变异系数（std/mean）:可表示客户的生活状态稳定性


## 数据理解
共有8张表：
1. 信用卡：card
1. 客户信息表：clients
1. 账户表：accounts
1. 贷款表：loans
1. 人口地区统计表：district
1. 权限分配表：disp
1. 支付命令表：order
1. 交易表：trans

In [1]:
import pathlib
import pandas as pd
import datetime
import numpy as np
import warnings
warnings.filterwarnings("ignore")

### 导入数据

In [2]:
folder = pathlib.Path.cwd().joinpath('CreditCard_Loan')

createVars = locals()
for fp in folder.iterdir():
    if fp.match('*.csv'):
        varname = fp.parts[-1].split('.')[0]
        createVars[varname] = pd.read_csv(fp, encoding='gbk')
        print(varname)

card
clients
accounts
loans
district
disp
order
trans


In [3]:
print('账户表有{}条记录'.format(accounts.shape[0]))
accounts.head(3)

# account_id 账户号，主键
# district_id 开户分行地区号，外键
# frequency 结算频度（交易之后马上、月结、周结）
# date 开户日期

账户表有4500条记录


,account_id,district_id,frequency,date
0,576,55,月结,1993-01-01
1,3818,74,月结,1993-01-01
2,704,55,月结,1993-01-01


In [4]:
print('客户信息表有{}条记录'.format(clients.shape[0]))
clients.head(3)

# client_id 客户号
# sex 性别
# birth_date 出生日期
# district_id 地区号（客户所属地区）

客户信息表有5369条记录


,client_id,sex,birth_date,district_id
0,1,女,1970-12-13,18
1,2,男,1945-02-04,1
2,3,女,1940-10-09,1


In [5]:
print('贷款表有{}条记录'.format(loans.shape[0]))
loans.head(2)

# loan_id 贷款号（主键）
# account_id 账户号（外键）
# date 发放贷款日期
# amount 贷款金额，整数，单位$，无逗号分隔
# duration 贷款期限
# payments 每月归还额，整数，单位$，无逗号分隔
# status 还款状态：A表合同终止，没问题；B表示合同终止，贷款没有支付；C表合同处于执行期，至今正常；D表示合同处于执行期，欠债状态。

贷款表有682条记录


,loan_id,account_id,date,amount,duration,payments,status
0,5314,1787,1993-07-05,96396,12,8033,B
1,5316,1801,1993-07-11,165960,36,4610,A


In [6]:
print('交易表有{}条记录'.format(trans.shape[0]))
trans.head(2)

# 交易表（Trans）：每条记录代表每个账户（account_id）上的一条交易
# trans_id 交易序号（主键）
# account_id 发起交易的账户号（外键）
# date 交易日期
# type 借贷类型：借（表示支出）、贷（表示收入）
# operation 交易类型：从他行收款、信用卡借方、信贷资金、汇款到另一家银行、现金
# amount 金额：$和逗号分隔
# balance 账户余额：$和逗号分隔
# k_symbol 交易特征：保险费、养老金、利息所得、房屋贷款、支付贷款、支票
# bank 对方银行：没加银行用两个字母代表
# account 对方账户号

交易表有1056320条记录


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,695247,2378,1993-01-01,贷,信贷资金,$700,$700,NaN,NaN,NaN
1,171812,576,1993-01-01,贷,信贷资金,$900,$900,NaN,NaN,NaN


In [7]:
print('权限分配表有{}条记录'.format(disp.shape[0]))
disp.head(2)

# 权限分配表(Disp) 每条记录描述了客户和账户之间的关系，以及客户操作账户的权限
# disp_id 权限设置号
# client_id 顾客号
# account_id 账户号
# type 权限类型：所有者和用户。所有者权限用户可以进行增值业务操作和贷款

权限分配表有5369条记录


,disp_id,client_id,account_id,type
0,1,1,1,所有者
1,2,2,2,所有者


In [8]:
print('信用卡表有{}条记录'.format(card.shape[0]))
card.head(2)

# 信用卡(Card) 每条记录描述了一个账户上的信用卡信息 
# card_id 信用卡ID（主键）
# disp_id 账户权限号（外键）
# issued 发卡日期
# type 卡类型：青年卡、金卡、普通卡

信用卡表有892条记录


,card_id,disp_id,issued,type
0,1005,9285,1993-11-07,普通卡
1,104,588,1994-01-19,普通卡


In [9]:
print('居住地区统计表有{}条记录'.format(district.shape[0]))
district.head(2)

# 人口地区统计表（District）：每条记录描述了一个地区的人口统计学信息
# A1 同district_id 地区号（主键）
# GDP	GDP总量
# A4	居住人口
# A10	城镇人口比例
# A11	平均工资
# A12	1995年失业率
# A13	1996年失业率
# A14	1000人中有多少企业家
# A15	1995犯罪率（千人）
# a16	1996犯罪率（千人）

居住地区统计表有77条记录


,A1,GDP,A4,A10,A11,A12,A13,A14,A15,a16
0,1,283894,1204953,100.0,12541,0.29,0.43,167,35.6,41.1
1,2,11655,88884,46.7,8507,1.67,1.85,132,12.1,15.0


## 数据准备与处理
### 创建预测变量Y：被解释变量

已知贷款表中的贷款状态有4类，分别为
A:合同终止，没问题；B:合同终止，贷款没有支付；C:合同执行期，至今正常；D：合同执行期，欠债状态。

In [10]:
# 将C类设为待定，即待预测
# A类为负样本即守约样本，B类C类为正样本即违约样本
loan_status = {'A':0,'C':2,'B':1,'D':1}
loans['bad_good'] = loans['status'].map(loan_status)

In [11]:
# 不同贷款状态的贷款数统计
loans.groupby(by='status').size()

status
A    203
B     31
C    403
D     45
dtype: int64

In [12]:
loans.head()

,loan_id,account_id,date,amount,duration,payments,status,bad_good
0,5314,1787,1993-07-05,96396,12,8033,B,1
1,5316,1801,1993-07-11,165960,36,4610,A,0
2,6863,9188,1993-07-28,127080,60,2118,A,0
3,5325,1843,1993-08-03,105804,36,2939,A,0
4,7240,11013,1993-09-06,274740,60,4579,A,0


### 选取特征变量X：解释变量
#### 客户表征信息
借款人的基本信息：借款时年龄、性别、账号级别信息等

In [13]:
data1 = pd.merge(loans, disp, on = 'account_id',how = 'left')
client_disp = pd.merge(data1, clients, on = 'client_id',how = 'left')

# 日期字符串转换为datatime类型
client_disp['date'] = pd.to_datetime(client_disp['date'])
client_disp['birth_date'] = pd.to_datetime(client_disp['birth_date'])#将字符串转换为datatime时间序列
# 计算借款时年龄
client_disp['age'] = round((client_disp['date']-client_disp['birth_date'])/datetime.timedelta(days=365),0)
client_disp.head() 

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,type,sex,birth_date,district_id,age
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,所有者,女,1947-07-22,30,46.0
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,所有者,男,1968-07-22,46,25.0
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,所有者,男,1936-06-02,45,57.0
3,5325,1843,1993-08-03,105804,36,2939,A,0,2235,2235,所有者,女,1940-04-20,14,53.0
4,7240,11013,1993-09-06,274740,60,4579,A,0,13231,13539,所有者,男,1978-09-07,63,15.0


#### 客户状态信息
借款人的居住地的经济状况：如居住地失业率、人口、GDP、平均工资、犯罪率等

In [14]:
client_loan_district = pd.merge(client_disp,district,left_on = 'district_id',right_on ='A1',how = 'left')
client_loan_district.head(3)

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A1,GDP,A4,A10,A11,A12,A13,A14,A15,a16
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,...,30,16979,94812,81.8,9650,3.38,3.67,100,15.7,14.8
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,...,46,14111,112709,73.5,8369,1.79,2.31,117,12.7,11.6
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,...,45,12888,77917,53.5,8390,2.28,2.89,132,13.3,13.6


#### 客户行为信息（资产状况）
客户的账户变动信息：计算如平均余额、余额波动、余额变异系数、平均入账与平均支出的比例、贷存比等，用于考察借款人的还款能力

In [15]:
# 借款人借款日期以及历史交易记录
trans_loan = pd.merge(loans[['account_id','date']], trans[['account_id','type','amount','balance','date']], on = 'account_id')
trans_loan.rename(columns={'date_x':'loan_date','date_y':'tran_date'}, inplace=True)
# 基于交易日期排序
trans_loan = trans_loan.sort_values(by =['account_id','tran_date'])
# 日期字段更改字符串类型为日期类型
trans_loan['loan_date'] = pd.to_datetime(trans_loan['loan_date'])
trans_loan['tran_date'] = pd.to_datetime(trans_loan['tran_date'])

# 金额字段]:去掉$符号和逗号分隔,提取数值金额
trans_loan['amount_v'] = trans_loan['amount'].map(lambda x: int(''.join(x[1:].split(','))) )
trans_loan['balance_v'] = trans_loan['balance'].map(lambda x: int(''.join(x[1:].split(','))) )

* 取数时间窗口 
鉴于我们要预测借款人在放贷后的违约概率，用户在放贷前一年的行为数据更有参考价值。
在这里，将贷前审批的**数据观测时间窗口**定为一年。

In [16]:
# 选出放款日期前1年（365天）至前1天的交易记录
print('借款人交易记录共有{}条。'.format(trans_loan.shape[0]))
trans_loan = trans_loan[(trans_loan['loan_date'] > trans_loan['tran_date'] ) 
                    & (trans_loan['loan_date'] < (trans_loan['tran_date'] + datetime.timedelta(days=365)))]
print('取数窗口为放款日期前1年（365天）至前1天，筛选出{}条记录。'.format(trans_loan.shape[0]))
trans_loan.head()

借款人交易记录共有191556条。
取数窗口为放款日期前1年（365天）至前1天，筛选出45366条记录。


,account_id,loan_date,type,amount,balance,tran_date,amount_v,balance_v
10020,2,1994-01-05,贷,"$1,100","$1,100",1993-02-26,1100,1100
10021,2,1994-01-05,贷,$20236,$21336,1993-03-12,20236,21336
10022,2,1994-01-05,贷,"$3,700",$25036,1993-03-28,3700,25036
10023,2,1994-01-05,贷,$14,$25050,1993-03-31,14,25050
10024,2,1994-01-05,贷,$20236,$45286,1993-04-12,20236,45286


* 计算借款人的账户余额信息：平均余额、余额波动、余额变异系数

In [17]:
# 平均账户余额（代表财富水平）、账户余额的标准差（代表财富稳定情况）和变异系数（代表财富稳定的另一个指标）。
balance_info = trans_loan.groupby(by='account_id')['balance_v'].agg([('avg_balance','mean'),('stdev_balance','std')]).reset_index()
balance_info['cv_balance'] =balance_info.apply(lambda x:x['stdev_balance']/x['avg_balance'], axis=1) 
balance_info.head(3)

,account_id,avg_balance,stdev_balance,cv_balance
0,2,32590.759259,12061.802206,0.370099
1,19,25871.223684,15057.521648,0.582018
2,25,56916.984496,21058.667949,0.369989


* 计算平均入账vs平均支出
在用户交易表（trans表）中，交易类型有2种，“贷”表收入，“借”表支出。

In [18]:
# 账户的收入与支出的汇总
income_out = trans_loan.groupby(by=['account_id','type'])['amount_v'].sum().reset_index()
income_out['type'] = income_out.type.map({'借':'out', '贷':'income'})
# 列转行，将每个账户的两条记录转换为每个账户一条记录
income_out = pd.pivot_table(income_out, values = 'amount_v', index = 'account_id',columns='type')

# 查看是否存在缺失值, 在支出列存在3个缺失值，进行缺失值填充
print('缺失值查看\n',income_out.isna().sum())
income_out.fillna(0, inplace=True)

# 计算支出占收入占比
income_out['rate_out_in'] = income_out.apply(lambda x: x['out']/x['income'], axis=1)
income_out.head(4)

缺失值查看
 type
income    0
out       3
dtype: int64


type,income,out,rate_out_in
account_id,,,
2,276514.0,153020.0,0.553390
19,254255.0,198020.0,0.778824
25,726479.0,629108.0,0.865969
37,386357.0,328541.0,0.850356


* 合并贷款用户贷款信息、客户基本信息、账户余额信息、入账支出信息

In [19]:
data = pd.merge(client_loan_district, balance_info,  on="account_id", how="left")
data = pd.merge(data, income_out, on="account_id", how="left")
data.head(3)

,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A13,A14,A15,a16,avg_balance,stdev_balance,cv_balance,income,out,rate_out_in
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,...,3.67,100,15.7,14.8,12250.000000,8330.866301,0.680071,20100.0,0.0,0.000000
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,...,2.31,117,12.7,11.6,43975.810811,25468.748605,0.579154,243576.0,164004.0,0.673318
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,...,2.89,132,13.3,13.6,30061.041667,11520.127013,0.383224,75146.0,54873.0,0.730219


* 计算贷存比、贷收比
如果贷款的金额远远高于自己的收入，就有可能最后还不上，所以需要求出货存比与贷收比来判断

In [20]:
# 贷存比 = 贷款项/平均资产
data['rate_loan_balance'] = data[['amount','avg_balance']].apply(lambda x: x[0]/x[1], axis=1)
# 贷收比 = 贷款项/平均收入
data['rate_loan_income'] = data[['amount','income']].apply(lambda x: x[0]/x[1], axis=1)
print('样本总量：',data.shape[0])
data.head(3)

样本总量： 827


,loan_id,account_id,date,amount,duration,payments,status,bad_good,disp_id,client_id,...,A15,a16,avg_balance,stdev_balance,cv_balance,income,out,rate_out_in,rate_loan_balance,rate_loan_income
0,5314,1787,1993-07-05,96396,12,8033,B,1,2166,2166,...,15.7,14.8,12250.000000,8330.866301,0.680071,20100.0,0.0,0.000000,7.869061,4.795821
1,5316,1801,1993-07-11,165960,36,4610,A,0,2181,2181,...,12.7,11.6,43975.810811,25468.748605,0.579154,243576.0,164004.0,0.673318,3.773893,0.681348
2,6863,9188,1993-07-28,127080,60,2118,A,0,11006,11314,...,13.3,13.6,30061.041667,11520.127013,0.383224,75146.0,54873.0,0.730219,4.227398,1.691108


## 数据清洗及特征选取

In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 827 entries, 0 to 826
Data columns (total 33 columns):
loan_id              827 non-null int64
account_id           827 non-null int64
date                 827 non-null datetime64[ns]
amount               827 non-null int64
duration             827 non-null int64
payments             827 non-null int64
status               827 non-null object
bad_good             827 non-null int64
disp_id              827 non-null int64
client_id            827 non-null int64
type                 827 non-null object
sex                  827 non-null object
birth_date           827 non-null datetime64[ns]
district_id          827 non-null int64
age                  827 non-null float64
A1                   827 non-null int64
GDP                  827 non-null int64
A4                   827 non-null int64
A10                  827 non-null float64
A11                  827 non-null int64
A12                  818 non-null float64
A13                  827 non

### 缺失值处理
* 查看存在数据缺失的字段

In [22]:
na = data.isnull().sum()/len(data)
na = na.reset_index()
na.rename(columns={0:'rate'}, inplace=True)
na[na['rate'] > 0]

,index,rate
20,A12,0.010883
23,A15,0.010883


1995年失业率A12、1995年犯罪率A15各存在1%的数据缺失，连续型数据类型可用中位数填充

In [23]:
data['A12'].fillna(data['A12'].median(), inplace=True)

### 变量梳理

In [24]:
#因变量
y = 'bad_good'

#连续变量：
var_c = ['amount', 'duration', 'payments', 'age', 
        'GDP', 'A4', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'a16',  
        'avg_balance','stdev_balance', 'cv_balance', 
        'income', 'out', 'rate_out_in', 
        'rate_loan_balance', 'rate_loan_income']

# 分类变量：性别 
var_d = ['sex']

In [25]:
# 二值化性别变量
data['sex'] = data.sex.map({'男':1,'女':0})

变量相关性查看

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt

corr = data[var_c + var_d].corr()
plt.figure(figsize=(13,10))
sns.heatmap(corr, vmax=1, annot=True, cmap='PiYG')
plt.show()

<Figure size 1300x1000 with 2 Axes>

### 特征变量调整
由热力图所示，发现很多变量之间存在高度相关性。需要在各信息类型种有选择的选取特征变量。
* 用户贷款状况：贷款金额(`amount`)与每月归还额(`payments`)呈现较强的正相关性，所以保留变量`amount`、`duration`;
* 用户居住地区状况：
	*反应地区的经济状况的GDP总量与平均工资（`A11`）、1000人中有多少企业家（`A14`）、1995犯罪率(`A15`)、1996犯罪率（`a16`）呈高度正相关。所以保留GDP或者计算人均GDP来进行替代;
	* 1995年失业率（`A12`）、1996年失业率（`A13`）与1000人中有多少企业家（`A14`）是呈现较强的负相关关系，有必要保留失业率信息，可以通过计算失业增长率来体现：1996年失业率/1995年失业率;
* 借款人经济状况：
	* 收入与支出呈现高度相关性，所以使用收入支出比（`rate_out_in`)这个变量来反应**客户的消费水平**；
	* 资产余额的波动情况用之前计算好的标准差(`stdev_balance`)、变异系数变量（`cv_balance`）来表示。

In [27]:
data['avg_gdp'] = data[['GDP','A11']].apply(lambda x: x[0]/x[1], axis= 1)
data['rate_unemploy_increase'] =  data[['A12','A13']].apply(lambda x: x[1]/x[0], axis= 1)

In [28]:
#因变量
y = 'bad_good'

#变量
var = ['amount', 'duration',                           # 贷款信息
        'age', 'sex',                                  # 贷款时年龄、贷款人性别
        'avg_gdp', 'rate_unemploy_increase', 'A14',    # 居住地人均GDP,失业增长率,千人企业家数
        'stdev_balance', 'cv_balance', 'rate_out_in',  # 资产余额（生活稳定性），收支比（消费水平）
        'rate_loan_balance', 'rate_loan_income'        # 贷存比，贷收比
        ]

* 查看调整后的变量相关性

In [29]:
corr = data[var].corr()
plt.figure(figsize=(6,5))
sns.heatmap(corr, vmax=1, annot=True, cmap='PiYG')
plt.show()

<Figure size 432x360 with 2 Axes>

## 建立分析模型
此处使用常用的逻辑回归模型做二分类预测。
### 制作训练集与测试集数据

In [30]:
selected_data = data[var+[y]]

# C类状态为合同还在执行期且还未出现逾期的借款人样本，提取C类用于预测
# 剩余样本用于模型训练与测试
# 无违约行为：'A':0，有违约行为：'B':1,'D':1，待定：'C':2
data_model = selected_data[selected_data.bad_good != 2]
data_predict = selected_data[selected_data.bad_good == 2]

print(data_predict.shape[0],'个账户待定')

493 个账户待定


**样本平衡性检查**
用于模型训练与测试的数据样本量：
* 负样本 0（贷款还款正常）：258个
* 正样本 1（贷款还款违约）：76个

正样本占整体样本的23%

In [31]:
data_model.groupby(by = [y]).size()

bad_good
0    258
1     76
dtype: int64

#### 方式一：整体随机抽样制作训练集与测试集
缺点：训练集与测试集中正负样本的比例并不一定和原始分布一致，正样本占总体样本共23%。如果仅是随机抽样这很大概率会出现训练集与测试集中正样本的占比过多或过小，造成训练集模型在训练学习的时候没能更好的学习到正样本的特征造成欠拟合，反之则过拟合。
改善：手动平衡分布

In [32]:
# from sklearn.model_selection import train_test_split

# X = np.array(data_model[var])
# Y = np.array(data_model[[y]])

# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state = 22)

# print("训练集样本量：{},测试集样本量{}".format(x_train.shape[0], x_test.shape[0]))

#### 方式二：手动平衡训练集与测试集的正负样本比例

In [33]:
import random

selected_test_sample_indices = {}
selected_train_sample_indicies = {}

for label in range(2):
    label_data = data_model[data_model[y] == label]
    selected_test_sample_indices[label] = random.sample(set(label_data.index), int(len(label_data)*0.3))

for label in range(2):
    label_data = data_model[data_model[y] == label]
    selected_train_sample_indicies[label] = list(set(label_data.index)- set(selected_test_sample_indices[label]))
    

In [34]:
train_data_indicies, test_data_indicies = [],[]

# 训练集的所有类别下标
for label,indicies in selected_train_sample_indicies.items():
    train_data_indicies = train_data_indicies + indicies # merge 2 list
# 测试集的所有类别下标
for label,indicies in selected_test_sample_indices.items():
    test_data_indicies = test_data_indicies + indicies

In [35]:
y_train = np.array(data_model[data_model.index.isin(train_data_indicies)][[y]])
x_train = np.array(data_model[data_model.index.isin(train_data_indicies)][var])

y_test = np.array(data_model[data_model.index.isin(test_data_indicies)][[y]])
x_test = np.array(data_model[data_model.index.isin(test_data_indicies)][var])

print("训练集样本量：{},测试集样本量{}".format(x_train.shape[0], x_test.shape[0]))

训练集样本量：235,测试集样本量99


## 模型训练
此处，我们采用L1正则化来减缓过拟合的问题，L1会保留最重要的变量。关于L1与L2的区别可以参考：[L1/L2正则化](https://zhuanlan.zhihu.com/p/25707761)

In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict # 交叉验证
from sklearn.metrics import accuracy_score


lr = LogisticRegression(penalty='l1')
predictions = cross_val_predict(lr, x_train, y_train, cv=10) # Kfold = 10(交叉验证)
print('训练集Accuracy Score: %.2f' % accuracy_score(y_train, predictions))


clf = lr.fit(x_train,y_train) # 开始训练
y_pred = clf.predict(x_test)  # 开始预测

# 模型的表现
print('Score: %.2f' % clf.score(x_test, y_test))

训练集Accuracy Score: 0.88
Score: 0.88


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

查看在决策函数中各特征变量的系数，发现申请贷款前一年的贷存比（`rate_loan_balance`）、贷款期限（`duration`）、贷款年龄（`age`）、资产变异系数（`cv_balance`）、资产余额标准差（`stdev_balance`）与违约**正相关**。贷款金额(`amount`）、人均GDP（`avg_gdp`）、贷款者当地1000人中有多少企业家（`A14`）、贷款者当地失业增长率（`rate_unemploy_increase`)、收支比（`rate_out_in`）与违约**负相关**。

In [37]:
coef = clf.coef_[0]
var_coef = dict(zip(var, coef))
sorted_var_coef = {k: v for k, v in sorted(var_coef.items(), key=lambda x: x[1])}

for x in sorted_var_coef.items():
    print('变量:{} 系数:{}'.format(x[0],x[1]))

变量:rate_out_in 系数:-0.7672370629849472
变量:rate_unemploy_increase 系数:-0.2053521331258804
变量:sex 系数:-0.10729222147860155
变量:A14 系数:-0.028963021027650346
变量:rate_loan_income 系数:-0.011233679321989735
变量:amount 系数:-1.8009923415493877e-05
变量:cv_balance 系数:0.0
变量:stdev_balance 系数:0.00017221390881922016
变量:age 系数:0.017540577793697023
变量:duration 系数:0.04591489790110804
变量:avg_gdp 系数:0.062164711051647725
变量:rate_loan_balance 系数:0.9962522129976491


## 模型效果评估

模型的精确率（预测正样本预测的准确度），召回率（查全率），f1_score如图所示。
对于不平衡样本数据集，是不建议用accuracy来作为模型效果评估指标的。
业务关注的侧重点会在TPR真正率和FPR假正率，违约的查全率。

In [38]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=[6,6])
plt.plot(fpr, tpr, 'r--', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
plt.title("ROC曲线")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend(loc="lower right")
plt.show()

              precision    recall  f1-score   support

           0       0.91      0.94      0.92        77
           1       0.75      0.68      0.71        22

    accuracy                           0.88        99
   macro avg       0.83      0.81      0.82        99
weighted avg       0.88      0.88      0.88        99



<Figure size 432x432 with 1 Axes>

In [39]:
print("模型的ROC曲线离左上角还是有段距离，曲线下面积（AUC）为{:.1f}%，\
说明模型的分类效果一般(0.85以上表示良好)，有改进提升空间。\n\
可以考虑对不平衡的正负样本量做平衡处理，或者对模型进行更多的参数调整。".format(100*roc_auc))

模型的ROC曲线离左上角还是有段距离，曲线下面积（AUC）为80.8%，说明模型的分类效果一般(0.85以上表示良好)，有改进提升空间。
可以考虑对不平衡的正负样本量做平衡处理，或者对模型进行更多的参数调整。


## 模型参数调整（数据增强）

这里会进行数据增强操作，考虑到通过新增造假数据来进行数据增强不现实也不合理，所以会通过调整正负样本的权重占比来实现。
提高占为22.7%的负样本的权重。正样本：负样本 =~ 1:4

In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict # 交叉验证
from sklearn.metrics import accuracy_score

# 数据增强，提高占为22.7%的负样本的权重。正样本：负样本 =~ 1:4
# 正样本(违约)权重：，负样本(正常)权重
c_w = {1:3, 0:1}

lr = LogisticRegression(penalty='l1', class_weight=c_w)
predictions = cross_val_predict(lr, x_train, y_train, cv=10) # Kfold = 10(交叉验证)
print('训练集Accuracy Score: %.2f' % accuracy_score(y_train, predictions))


clf = lr.fit(x_train,y_train) # 开始训练
y_pred = clf.predict(x_test)  # 开始预测

print('模型Score: %.2f' % clf.score(x_test, y_test))

训练集Accuracy Score: 0.82
模型Score: 0.83


/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

现在再来查看在决策函数中各特征变量的系数，发现申请贷款前一年的
- 资产变异系数（`cv_balance`）、贷存比（`rate_loan_balance`）、贷收比（`rate_loan_income`）、人均GDP（`avg_gdp`）、贷款期限（`duration`）、贷款年龄（`age`）、资产余额标准差（`stdev_balance`）与违约**正相关**。
- 贷款金额(`amount`）、贷款者当地1000人中有多少企业家（`A14`）、收支比（`rate_out_in`）与违约**负相关**。

调整后的模型的变量系数看起来更合理，也符合预期。

In [41]:
coef = clf.coef_[0]
var_coef = dict(zip(var, coef))
sorted_var_coef = {k: v for k, v in sorted(var_coef.items(), key=lambda x: x[1])}

for x in sorted_var_coef.items():
    print('变量:{} 系数:{}'.format(x[0],x[1]))

变量:rate_out_in 系数:-1.1612272948937754
变量:rate_unemploy_increase 系数:-0.20077021498496475
变量:sex 系数:-0.09431822022948862
变量:A14 系数:-0.01836468715967477
变量:amount 系数:-9.644899315057778e-06
变量:stdev_balance 系数:7.830858271106624e-05
变量:age 系数:0.02108231103143271
变量:avg_gdp 系数:0.03348604817465702
变量:duration 系数:0.04403274686369934
变量:rate_loan_income 系数:0.04572509638689259
变量:rate_loan_balance 系数:0.6010459530475475
变量:cv_balance 系数:5.179685255615256


## 调整后模型效果评估

In [42]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred)) 
  
  
from sklearn.metrics import roc_curve, auc

fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=[6,6])
plt.plot(fpr, tpr, 'r--', label='ROC curve (area = {:.2f})'.format(roc_auc))
plt.plot([0, 1], [0, 1], color='navy',  linestyle='--')
plt.title("ROC曲线")
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend(loc="lower right")
plt.show()

              precision    recall  f1-score   support

           0       0.94      0.83      0.88        77
           1       0.58      0.82      0.68        22

    accuracy                           0.83        99
   macro avg       0.76      0.82      0.78        99
weighted avg       0.86      0.83      0.84        99



<Figure size 432x432 with 1 Axes>

AUC有了提升，但距离良好的模型效果依然有优化空间。

## 模型运用
贷款状态为C的账户是尚没有出现违约且合同未到期的客户。这些贷款客户中有些人的违约可能性较高，需要业务人员重点关注。可以及早发现问题，及时处理，挽回损失。

In [43]:
data_predict["class"] = clf.predict(data_predict[var])
data_predict["违约预测"] = data_predict["class"].map({1:'会',0:'不会'})


for_predict_accountid_loanid = data[data.index.isin(data_predict.index)][['loan_id','account_id']]
predict_result = pd.merge(for_predict_accountid_loanid, data_predict, left_on = for_predict_accountid_loanid.index, right_on= data_predict.index, right_index = True, how='inner')

class1 = predict_result[predict_result["class"]==1][["account_id","loan_id","class","违约预测"]]
class1_account = class1.account_id.unique()
print('待定贷款共{}个,其中{}个贷款有违约可能，涉及到{}个账户。'.format(data_predict.shape[0], data_predict[data_predict['class']== 1].shape[0], len(class1_account)))

待定贷款共493个,其中251个贷款有违约可能，涉及到207个账户。
